# Dependencies

In [363]:
from langchain_community.document_loaders import PyPDFLoader,TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter 
from openai import OpenAI
import requests

from langchain_openai import ChatOpenAI
from langchain.chains import LLMChain
from pinecone import Pinecone, ServerlessSpec
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate


from langchain.agents import initialize_agent, Tool
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage,BaseMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

from langchain.agents.format_scratchpad.openai_tools import (
    format_to_openai_tool_messages,
)
from langchain.agents.output_parsers.openai_tools import OpenAIToolsAgentOutputParser
from langchain.agents import AgentExecutor

from langchain_core.tools import tool
from pydantic.v1 import BaseModel, Field
from openai import OpenAI


from typing import Literal
from typing_extensions import TypedDict

from langgraph.graph import MessagesState, END
from langgraph.types import Command

from typing import Literal
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, MessagesState, START, END

from pydantic import BaseModel, Field, ValidationError
from typing import List, Union, Dict, Optional
from pinecone import Index

import validators
import requests
import os
import uuid

from dotenv import load_dotenv
load_dotenv()

import os
import sys
import time

# CrewAI
from crewai import Agent, Task, Crew, Process
from textwrap import dedent


OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
PINECONE_API_ENV = os.getenv("PINECONE_API_ENV")
PINECONE_QA_BASE_INDEX_NAME = os.getenv("PINECONE_QA_BASE_INDEX_NAME")
PINECONE_KNOWLEDGE_BASE_INDEX_NAME = os.getenv("PINECONE_KNOWLEDGE_BASE_INDEX_NAME")

In [131]:
openAI_client = OpenAI(api_key=OPENAI_API_KEY)
embedding_model = openAI_client.embeddings
pc = Pinecone(api_key = PINECONE_API_KEY, environment = PINECONE_API_ENV)
doc_index = pc.Index(PINECONE_KNOWLEDGE_BASE_INDEX_NAME)
qa_index = pc.Index(PINECONE_QA_BASE_INDEX_NAME)


def get_embedding(text) :
    """
        Function to convert the text string into embeddings using text-embedding-3-small from OpenAI
    
        Args:
            text : A string which will contain either the text chunk or the user query
            
        Returns:
            vector : A vector of 1536 dimensions
    """
    
    try:
        response = embedding_model.create(
            input=text,
            model="text-embedding-3-small"
        )
        
        return response.data[0].embedding   
    
    except Exception as e:
        raise Exception(str(e))
    

# Agents and Tools Setup
Here, we need the worker agents and tools setup for this to work and we will be using the following : 

- Text Knowledge Base Agent
- Q&A Knowledge Base Agent
- Fallback Agent

For this, we will be using CrewAI. 

## Text Knowledge Base Agent

In [385]:
kb_query_prompt_template = """
    \n\n User Query : {query}
    \n\n Documents : {documents}
"""

kb_query_prompt = PromptTemplate(
    input_variables=["query","documents"],
    template= kb_query_prompt_template
)

kb_system_instructions = """
    You are a specialised AI document analyser working at an edtech startup, and you will be assisting the users to answer their queries. You will be given 
    the top relevant documents and you have to use those to answer the query asked by the user, which will be given to you below. 
    
    In the relevant documents,you will be given the cosine similarity score, the document name and the reference (which is the page number where this 
    text was in the document) and the text itself. You can in you answer integrate the document name and reference to build authenticity of your answer, 
    by precisely writing it like (reference page : page_num). You should eventually tell the user to explore more about the topic in the document and that 
    reference page. Infact, this would be highly favourable if you mention the document name and reference to build your authenticity, however do not be 
    repetitive in giving references, just give it once in the answer. 
    
    MAKE SURE YOU DO NOT ANSWER FROM ANYTHING APART FROM THE DOCUMENTS GIVEN TO YOU. IF THE DOCUMENTS DO NOT CONTAIN THE ANSWER, THEN YOU CAN SAY THAT
    I AM UNABLE TO FIND THE ANSWER IN THE DOCUMENTS GIVEN TO ME OR "I DO NOT KNOW" STRAIGHT AWAY. DO NOT TRY TO GUESS THE ANSWER WITHOUT THE DOCUMENTS,
    YOUR ANSWER SHOULD BE BASED ON THE DOCUMENTS GIVEN TO YOU AND BE PROPERLY REFERENCING THE DOCUMENT NAME AND REFERENCE PAGE, ELSE YOU CAN SAY I DO NOT KNOW.  
"""


In [386]:
def kb_fetch_answer(final_prompt):
    response = openAI_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": kb_system_instructions
            },
            {
                "role": "user",
                "content": final_prompt
            }
        ]
    )
    
    return response.choices[0].message.content

In [387]:
@tool
def kb_answer_query(query):
    """
    Find relevant documents for a given query and answer the query using the relevant documents.
    
    Args:
    - query (str): The search query will be in string format which will be passed straight to the LLM to find the relevant documents.
    
    Returns:
        Final text answer from the LLM.
    """
    if not isinstance(query, str) or not query.strip():
        return "Invalid query. Please provide a valid search term."
    
    query_vector = get_embedding(query)  
    results = doc_index.query(  
        vector=query_vector,
        top_k=5,
        include_values=False,
        include_metadata=True,
        filter={"userID": os.getenv("ADMIN_ID")}
    )

    relevant_texts = []
    for record in results['matches']:
        text = {
            'score': record['score'],
            'text': record['metadata']['chunk'],
            'name': record['metadata']['document_name'],
            'reference': int(record["metadata"]["page_number"]) + 1
        }
        relevant_texts.append(text)
    
    if not relevant_texts:
        return "I do not know"
    
    final_prompt = kb_query_prompt.invoke({"query": query, "documents": relevant_texts}).text
    return kb_fetch_answer(final_prompt)
    

### Crew Setup via CrewAI

In [388]:
text_knowledge_base_agent = Agent(
    role = "Librarian",
    goal = "To provide answer to the user's query based on the information in knowledge base",
    backstory = """
            You are an assistant who can handle queries by using the tool you have and refer the user to correct material, you just have to pass the exact user query (which is : "{query}" ) to the tool as text and it will do the rest.
            You have to return the answer given by the tool AS IT IS if it gives an answer (as it might have references), however if the tool tells you that unfortunately it does not have the answer or the answer is unrelated, you have to simply return "I do not know".
            DO NOT TRY TO ANSWER YOURSELF, IF THE TOOL SAYS IT DOES NOT KNOW OR IS UNABLE TO FIND AN ANSWER, YOU HAVE TO SAY I DO NOT KNOW.
    """,
    memory = True,
    verbose = True,
    tools = [kb_answer_query]
)

In [389]:
task_kb = Task(
    description="Answer the user's query by passing only the raw query string to the tool.",
    expected_output="An answer to the user's query based on the information in the knowledge base.",
    agent=text_knowledge_base_agent,
    parameters={"query": "str"}
)

In [390]:
crew_kb = Crew(agents=[text_knowledge_base_agent], tasks=[task_kb])

In [391]:
query = "What does useState even mean?"
result = crew_kb.kickoff(inputs={"query": query})
print(result)

# Agent: Librarian
## Task: Answer the user's query by passing only the raw query string to the tool.


# Agent: Librarian
## Using tool: kb_answer_query
## Tool Input: 
"{\"query\": \"What does useState even mean?\"}"
## Tool Output: 
The term "useState" refers to a Hook in React that allows you to add state to functional components. It provides an array containing the current state and a function to update that state. This feature is crucial for managing state within components effectively (reference page: 1, document name: W9-react). To delve deeper into how "useState" works and see examples, you might want to explore the document W9-react further, particularly on the reference page mentioned.


# Agent: Librarian
## Final Answer: 
The term "useState" refers to a Hook in React that allows you to add state to functional components. It provides an array containing the current state and a function to update that state. This feature is crucial for managing state within components effect

In [392]:
query = "What is the difference between useState and useEffect?"
result = crew_kb.kickoff(inputs={"query": query})
print(result)

# Agent: Librarian
## Task: Answer the user's query by passing only the raw query string to the tool.


# Agent: Librarian
## Thought: I need to find the answer to the user's query about the difference between useState and useEffect in React.
## Using tool: kb_answer_query
## Tool Input: 
"{\"query\": \"What is the difference between useState and useEffect?\"}"
## Tool Output: 
The `useState` and `useEffect` hooks in React serve different purposes. 

- `useState` is used to add state to functional components. It returns an array with the current state value and a function to update it (reference page: 1, W9-react).

- `useEffect` allows you to perform side effects in functional components in a predictable way. It is often used for operations like data fetching, subscriptions, or manually changing the DOM, and can return a cleanup function to be executed when the component unmounts (reference page: 6, W9-react).

For a comprehensive understanding, you may refer to the document "W9-react

In [393]:
query = "I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error"
result = crew_kb.kickoff(inputs={"query": query})
print(result)

# Agent: Librarian
## Task: Answer the user's query by passing only the raw query string to the tool.


# Agent: Librarian
## Thought: I need to find the relevant information regarding the user's issue with the NEXTJS project and the hydration error while animating the dropdown menu.
## Using tool: kb_answer_query
## Tool Input: 
"{\"query\": \"I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error\"}"
## Tool Output: 
I am unable to find the specific answer to your issue with the hydration error in a NEXTJS project related to animating a dropdown menu in the provided documents. It seems like the documents are more focused on general React hooks and error handling, rather than specific issues with Next.js and animations. You may want to try exploring solutions in documentation or forums specifically addressing Next.js for more detailed troubleshooting.


You ONLY have access to the following tools, and should NEVER make up tools that 

## Q&A Knowledge Base Agent

In [394]:
system_instructions_qa = """
    You are a specialised AI context aware doubt solver working at an edtech startup, and you will be assisting the users to answer their queries based on previous intrcutors and TAs solved queries.
    You will be given a query by the user and the top relevant documents and you have to use those to answer the query asked by the user, which will be given to you below. 
    In the relevant documents,you will be given the id of the conversation, the cosine similarity score, the question which was aksed by previous student and the answers by the TAs, along with the id of the QA pair (QA_ID). 
    YOU MUST tell the user that they can explore this further by going to that thread (give them the QA_ID) and looking at the entire conversation for better understanding. (Think of this as a reference to build authenticity, as you mention the QA_ID).
    This QA_ID points to a thread in discord where the conversation was held. However do not be repetitive in giving references, just give it once at the end of the answer.
    
    MAKE SURE YOU DO NOT ANSWER FROM ANYTHING APART FROM THE DOCUMENTS GIVEN TO YOU. IF THE DOCUMENTS DO NOT CONTAIN THE ANSWER, THEN YOU CAN SAY THAT
    I AM UNABLE TO FIND THE ANSWER IN THE DOCUMENTS GIVEN TO ME OR "I DO NOT KNOW" STRAIGHT AWAY. DO NOT TRY TO GUESS THE ANSWER WITHOUT THE DOCUMENTS,
    YOUR ANSWER SHOULD BE BASED ON THE DOCUMENTS GIVEN TO YOU AND BE PROPERLY REFERENCING THE QA_ID, ELSE YOU CAN SAY I DO NOT KNOW.  
"""

query_prompt_template_qa = """
    \n\n User Query : {query}
    \n\n Documents : {documents}
"""

query_prompt_qa = PromptTemplate(
    input_variables=["query","documents"],
    template=query_prompt_template_qa
)

In [395]:
def fetch_answer_qa(final_prompt):
    response = openAI_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "developer",
                "content": system_instructions_qa
            },
            {
                "role": "user",
                "content": final_prompt
            }
        ]
    )
    
    return response.choices[0].message.content

In [396]:
@tool
def answer_query_qa(query):
    """
        Find relevant documents for a given query and answer the query using the relevant documents.
        
        Args:
        - query (str): The search query will be in string format which will be passed straight to the LLM to find the relevant documents.
        
        Returns:
            Final text answer from the LLM.
    """
    if not isinstance(query, str) or not query.strip():
        return "Invalid query. Please provide a valid search term."
    
    query_vector = get_embedding(query)
    
    results = qa_index.query(
        vector = query_vector,
        top_k = 10,
        include_values = False,
        include_metadata = True,
        filter={
            "ADMIN_ID" : "d359d72b-40e8-4e9f-b567-62d77f273113",
        }
    )
    
    relevant_pairs = []
    for record in results['matches']:
        pair = {}
        pair['id'] = record['metadata']['QA_ID']
        pair['score'] = record['score']
        pair['question'] = record['metadata']['question']
        pair['answer'] = record['metadata']['answer']
        relevant_pairs.append(pair)
    
    if not relevant_pairs:
        return "I do not know"
    
    final_prompt = query_prompt_qa.invoke({"query" : query,"documents": relevant_pairs}).text
    return fetch_answer_qa(final_prompt)
    

## Agent Setup by CrewAI

In [397]:
QA_knowledge_base_agent = Agent(
    role = "Resolver",
    goal = "To provide answer to the user's query based on the past solved threads in knowledge base",
    backstory = """
            You are an assistant who can handle queries by using the tool you have and refer the user to correct material, you just have to pass the exact user query (which is : "{query}" ) to the tool as text and it will do the rest.
            You have to return the answer given by the tool AS IT IS if it gives an answer (as it might have references), however if the tool tells you that unfortunately it does not have the answer or the answer is unrelated, you have to simply return "I do not know".
            DO NOT TRY TO ANSWER YOURSELF, IF THE TOOL SAYS IT DOES NOT KNOW OR IS UNABLE TO FIND AN ANSWER, YOU HAVE TO SAY I DO NOT KNOW RIGHT AWAY.
    """,
    memory = True,
    verbose = True,
    tools = [answer_query_qa]
)

In [398]:
task_qa = Task(
    description="Answer the user's query by passing only the raw query string to the tool.",
    expected_output="An answer to the user's query based on the information in the knowledge base.",
    agent=QA_knowledge_base_agent,
    parameters={"query": "str"}
)

In [399]:
crew_qa = Crew(agents=[QA_knowledge_base_agent], tasks=[task_qa])

In [400]:
query = "I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error."
result = crew_qa.kickoff(inputs={"query": query})
print(result)

# Agent: Resolver
## Task: Answer the user's query by passing only the raw query string to the tool.


# Agent: Resolver
## Thought: I need to find relevant information regarding the user's issue with the NEXTJS project, specifically about the animation of a dropdown menu causing hydration errors.
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"I was having issues in my NEXTJS project, as I tried to animate the dropdown menu, I got a hydration error.\"}"
## Tool Output: 
To resolve the hydration error in your Next.js project while trying to animate a dropdown menu, you can start by restarting the server and using the browser's inspect element tool to verify if Tailwind CSS is being loaded and applied correctly. If styles are not applied despite being shown, it might indicate a configuration issue with Tailwind CSS. You should check the Tailwind CSS documentation to ensure it is correctly set up in your project. This involves verifying your Tailwind configuration file and 

In [401]:
query = "I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Why did this happen?"
result = crew_qa.kickoff(inputs={"query": query})
print(result)

# Agent: Resolver
## Task: Answer the user's query by passing only the raw query string to the tool.


# Agent: Resolver
## Thought: I need to find relevant documents that address the user's question regarding the CORS issue resolved by removing a 'return' statement.
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"I encountered an issue with CORS in my application, and it was resolved by removing a 'return' statement. Why did this happen?\"}"
## Tool Output: 
The issue with CORS in your application was resolved by removing a 'return' statement likely because the 'return' statement interrupted the middleware execution flow. In this case, the CORS configuration, which is typically set up in middleware, might not have been applied correctly due to the premature termination of the function. Once the 'return' statement was removed, the CORS setup could execute fully, allowing the configuration to be correctly applied, which resolved the issue. 

For more detailed insights, you

In [402]:
query = "How do JWT tokens work?"
result = crew_qa.kickoff(inputs={"query": query})
print(result)

# Agent: Resolver
## Task: Answer the user's query by passing only the raw query string to the tool.


# Agent: Resolver
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"How do JWT tokens work?\"}"
## Tool Output: 
I am unable to find the answer to how JWT tokens work in the documents given to me. You might need to explore other resources or ask again for new information.


You ONLY have access to the following tools, and should NEVER make up tools that are not listed here:

Tool Name: answer_query_qa
Tool Arguments: {'query': {'description': None, 'type': 'Any'}}
Tool Description: Find relevant documents for a given query and answer the query using the relevant documents.

Args:
- query (str): The search query will be in string format which will be passed straight to the LLM to find the relevant documents.

Returns:
    Final text answer from the LLM.

IMPORTANT: Use the following format in your response:

```
Thought: you should always think about what to do
Action: the 

## FallBack Agent

In [403]:
fb_system_instructions = """
        You are a specialised AI fallback assistant, but you must realise that if a query comes to you, it has already crossed the knowledge bank  and still the user is asking, means they clearly need a very simple and easy to digest explanation. Try to keep it simple yet understandable.
        Try to help them with examples and annecdotes to make sure they understand. Although the questions you would be getting would be of the CS concept, but try to dumb them down for the user. 
        Do not say that you have access to any documents or other resources, just try to read the query and answer in simple terms.
        Hence, YOU MUST TELL AT THE START OF YOUR ANSWER CLEARLY THAT THIS IS AI GENERATED (even a tag like [AI GENERATED] telling the user with a warning will be optimal)
        In the end, YOU MUST TELL THEM that they can escalate their ticket to the TAs (Teacher Assistants) further from the sidebar in this chat. (You will be answering in a discord chat, so the users will have the option to escalate the ticket to the TAs).
"""

In [404]:
def fetch_answer_fb(final_prompt):
    response = openAI_client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "developer",
                "content": fb_system_instructions
            },
            {
                "role": "user",
                "content": final_prompt
            }
        ]
    )
    
    return response.choices[0].message.content

In [405]:
@tool
def answer_query_fb(query):
    """
        Answer the query in a simple and easy to understand manner.
    """
    if not isinstance(query, str) or not query.strip():
        return "Invalid query. Please provide a valid search term."
    
    return fetch_answer_fb(query)
    

In [406]:
FB_agent = Agent(
    role = "Fallback",
    goal = "To provide answer to the user's query based on the past solved threads in knowledge base",
    backstory = """
            You are an assistant who can handle queries by using the tool you have and refer the user to correct material, you just have to pass the exact user query (which is : "{query}" ) to the tool as text and it will do the rest.
            You have to return the answer given by the tool AS IT IS if it quotes a relevant answer, in most cases the tool will return a simple answer,
            so you can just return the answer as it is. However if the tool tells you that unfortunately it does not have the answer or the answer is unrelated, you have to simply return "I do not know"
            DO NOT TRY TO ANSWER YOURSELF, IF THE TOOL SAYS IT DOES NOT KNOW OR IS UNABLE TO FIND AN ANSWER, YOU HAVE TO SAY I DO NOT KNOW RIGHT AWAY.
    """,
    memory = True,
    verbose = True,
    tools = [answer_query_fb]
)

In [407]:
task_fb = Task(
    description="Answer the user's query by passing only the raw query string to the tool.",
    expected_output="An answer to the user's query",
    agent=FB_agent,
    parameters={"query": "str"}
)

In [408]:
crew_fb = Crew(agents=[FB_agent], tasks=[task_fb])

In [410]:
query = "can you just explain me what are promises in javascript?"
result = crew_fb.kickoff(inputs={"query": query})
print(result)

# Agent: Fallback
## Task: Answer the user's query by passing only the raw query string to the tool.


# Agent: Fallback
## Using tool: answer_query_fb
## Tool Input: 
"{\"query\": \"can you just explain me what are promises in javascript?\"}"
## Tool Output: 
[AI GENERATED]

Alright! Let's talk about promises in JavaScript in a super simple way.

Imagine you ordered a pizza. The pizza shop promises to deliver it to you. While you wait, you can do other things like watch TV or chat with a friend, instead of standing by the door waiting for the delivery. Once the pizza arrives, you get a call or a knock on the door, and then you deal with the pizza.

In JavaScript, a Promise is something like a pizza order. It's a way to handle things (like loading data) that take time. Instead of stopping everything until the task is done, JavaScript uses promises to keep working on other things and checks back on the task when it's finished.

A promise can be:
1. **Pending** (pizza hasn't arrived yet)

# Supervisor Agent Setup
Now, we will combine all these agents in a team and create a supervisor agent, which acts as a manager for all the agents
and will decide which agent to use based on the user query.

In [ ]:
from crewai import Agent, Task, Crew

supervisor_agent = Agent(
    role='Supervisor',
    goal='Route queries to agents and answer user queries',
    backstory="""
        You are strictly a supervisor agent. Any user query given to you must be routed to the agents in the following order:
        1. Resolver, if it returns 'unable to find' or 'I do not know', ONLY THEN route to Librarian, else return the answer to the user and stop the process.
        2. Librarian, if it returns 'unable to find' or 'I do not know', ONLY THEN route to Fallback, else return the answer to the user and stop the process.
        3. Fallback, if it returns 'unable to find' or 'I do not know', return 'Sorry, I am unable to answer at this moment' to the user.
        
        REMEMBER : NEVER ANSWER A QUERY YOURSELF, ALWAYS ROUTE TO THE AGENTS. UNDER NO CIRCUMSTANCES SHOULD YOU ANSWER THE QUERY YOURSELF OR GIVE A RESPONSE TO THE USER BY YOUR KNOWLEDGE.
      """,
    memory=True, 
    verbose=True,
    allow_delegation=True
)

crew = Crew(
    agents=[QA_knowledge_base_agent, text_knowledge_base_agent, FB_agent],
    tasks=[task_qa, task_kb, task_fb],
    process = Process.sequential,
    manager_agent = supervisor_agent
)


def process_query(query):
    try:
        result = crew.kickoff(inputs={"query": query})
        return result
    except Exception as e:
        print(f"Error: {str(e)}")
        return "An error occurred"

In [415]:
query = "nextJS hydration error"
result = process_query(query)
print(result)

# Agent: Supervisor
## Task: Answer the user's query by passing only the raw query string to the tool.
# Agent: Resolver
## Task: What is the answer to the user's query?


# Agent: Resolver
## Using tool: answer_query_qa
## Tool Input: 
"{\"query\": \"nextJS hydration error\"}"
## Tool Output: 
To resolve a hydration error in a Next.js project, especially if you're facing issues with disappearing styles after making changes, you can take the following steps:

1. Restart the server to see if that resolves the issue.
2. Use the browser's inspect element tool to check if Tailwind CSS is loading correctly. If styles are loaded but not applied, it might indicate a configuration issue with Tailwind CSS.
3. Verify your Tailwind CSS setup by checking the configuration file and ensuring it is correctly set up in your project. Make sure that the necessary classes are being generated and applied correctly.

For more detailed steps and context, you can explore this further by looking at the releva

KeyboardInterrupt: 